# Inbound flights × excess mortality — correlation results

M. Elixhauser

In [ ]:
source(here::here("R", "00_load_libs.R"))


## Analysis data set

In [ ]:
corr_df <- analysis_df |>
  semi_join(analysis_df |> count(iso3) |> filter(n == 4), by = "iso3") |> # Keep only countries with complete data (all 4 analysis dates)
  left_join(pop_df, by = "iso3") |>
  mutate(flights_pm = total_inbound_flights_combined / (population / 1e6))
stopifnot(!anyNA(corr_df$flights_pm))


## Spearman’s rank correlation (baseline)

In [ ]:
# Bootstrapped Spearman rho for robust CIs (idx is bootstrap sample indices)
boot_rho <- function(data, idx, xvar) {
  with(
    data[idx, ],
    cor(get(xvar),
      excess_mortality_cumulative_per_million,
      method = "spearman",
      use = "complete.obs"
    )
  )
}

spearman_by_year <- function(xvar, B = 5000) {
  corr_df |>
    group_by(target_date) |>
    group_modify(\(d, key) {
      ok <- complete.cases(
        d[[xvar]],
        d$excess_mortality_cumulative_per_million
      )
      dd <- d[ok, ]
      n <- nrow(dd)

      if (n < 3 ||
        sd(dd[[xvar]]) == 0 ||
        sd(dd$excess_mortality_cumulative_per_million) == 0) {
        return(tibble(
          rho = NA, ci_lo = NA, ci_hi = NA,
          p = NA, n = n
        ))
      }

      rho <- cor(dd[[xvar]], dd$excess_mortality_cumulative_per_million,
        method = "spearman"
      )
      pval <- cor.test(dd[[xvar]], dd$excess_mortality_cumulative_per_million,
        method = "spearman", exact = FALSE
      )$p.value

      set.seed(42)
      ci <- if (n >= 4) {
        boot(dd, boot_rho, R = B, xvar = xvar) |>
          (\(b) quantile(b$t, c(.025, .975), na.rm = TRUE))() # .025, .975 CI (why used!)
      } else {
        c(NA, NA)
      }

      tibble(
        rho = rho,
        ci_lo = ci[1],
        ci_hi = ci[2],
        p = pval,
        n = n
      )
    }) |>
    ungroup() |>
    mutate(
      variable = xvar,
      year = lubridate::year(target_date)
    )
}

spearman_res <- purrr::map_dfr(
  c(
    "total_inbound_flights_dec19",
    "total_inbound_flights_mar20",
    "total_inbound_flights_combined",
    "flights_pm"
  ),
  spearman_by_year
) %>%
  mutate(variable = dplyr::recode(variable,
    total_inbound_flights_dec19    = "Dec 2019",
    total_inbound_flights_mar20    = "Mar 2020",
    total_inbound_flights_combined = "Combined",
    flights_pm                     = "Flights / M pop"
  ))


In [ ]:
knitr::kable(
  spearman_res |>
    select(year, variable, rho, ci_lo, ci_hi, p, n) |>
    mutate(across(where(is.numeric), \(x) round(x, 3))),
  caption = "Table X Spearman ρ between early flight exposure and cumulative excess mortality."
)


    year variable               rho    ci_lo    ci_hi       p    n
  ------ ----------------- -------- -------- -------- ------- ----
    2020 Dec 2019             0.502    0.135    0.759   0.011   25
    2021 Dec 2019            -0.255   -0.591    0.161   0.218   25
    2022 Dec 2019            -0.391   -0.761    0.071   0.053   25
    2023 Dec 2019            -0.343   -0.713    0.119   0.093   25
    2020 Mar 2020             0.573    0.228    0.796   0.003   25
    2021 Mar 2020            -0.296   -0.644    0.135   0.151   25
    2022 Mar 2020            -0.441   -0.782    0.017   0.027   25
    2023 Mar 2020            -0.390   -0.734    0.064   0.054   25
    2020 Combined             0.526    0.177    0.771   0.007   25
    2021 Combined            -0.306   -0.622    0.111   0.137   25
    2022 Combined            -0.460   -0.792   -0.001   0.021   25
    2023 Combined            -0.419   -0.749    0.045   0.037   25
    2020 Flights / M pop      0.303   -0.077    0.606   0.141   25
    2021 Flights / M pop     -0.531   -0.787   -0.126   0.006   25
    2022 Flights / M pop     -0.621   -0.825   -0.261   0.001   25
    2023 Flights / M pop     -0.570   -0.775   -0.233   0.003   25

  : Table X Spearman ρ between early flight exposure and cumulative
  excess mortality.


**Key patterns:**

-   Early air connectivity explained a substantial share of cross-country variation in first-wave excess mortality (ρ ≈ 0.53, 95% CI \[0.18, 0.80\], p \< 0.01).
-   From 2021 onwards, the rank correlation essentially disappeared; by 2022–23 it became modestly negative (ρ ≈ –0.45, 95% CI \[–0.79, 0.00\]).
-   This sign change exceeds the bootstrap confidence intervals, suggesting the shift is unlikely due to random sampling in a ~25-country dataset.
-   Notably, **“Flights per million population” was already significantly negative from 2021 onward**, indicating that higher per-capita connectivity was linked to *lower* late-pandemic mortality—a possible “resource advantage” effect in highly connected countries.

**Interpretation:**

-   International flights played a major role only during the initial seeding phase—before widespread community transmission in Europe.
-   Once SARS-CoV-2 was established in most countries, *domestic factors*—population age, public health response, healthcare capacity, and vaccination coverage—became dominant in shaping excess mortality.
-   This pattern aligns with studies showing that the effect of border controls diminishes once local transmission is established (see @chinazzi2020).

**Why bootstrap CIs?**

-   We computed 5,000 bootstrap resamples, following best practice for small-n correlation studies (DiCiccio & Efron 1996; Tong et al. 2016; Mokhtar 2023). This provides stable confidence intervals, and a fixed random seed (`set.seed(42)`) ensures full reproducibility.

**Summary:**

Early in the pandemic, countries most exposed to direct flights from China and Hong Kong suffered higher excess mortality. However, as the pandemic progressed, this association disappeared—and by 2022–2023, even reversed: high connectivity was linked to *lower* excess deaths, likely reflecting differences in domestic response, health system capacity, and perhaps a “rich country” advantage. These findings underscore the narrow window during which travel restrictions can affect epidemic outcomes, and the overwhelming importance of internal factors once local transmission is widespread.

# Visualizing the trend

## p over time

In [ ]:
pal <- c(
  "Dec 2019" = "#E69F00",
  "Mar 2020" = "#56B4E9",
  "Combined" = "#009E73",
  "Flights / M pop" = "#CC79A7"
)

p_rho <- ggplot(
  spearman_res,
  aes(target_date, rho, colour = variable, group = variable)
) +
  geom_line(linewidth = 1.1) +
  geom_point(size = 2.4) +
  geom_hline(yintercept = 0, linetype = "dashed", colour = "grey30") +
  scale_colour_manual(values = pal, name = NULL) +
  scale_x_date(date_labels = "%Y") +
  labs(
    x = NULL,
    y = "Spearman ρ",
    colour = NULL
  ) +
  theme_minimal(base_size = 11) +
  theme(legend.position = "top")

p_rho |> (\(p) {
  ggsave(
    filename = here(fig_dir, "rho_timeseries.png"),
    plot = p,
    width = 6, height = 4, dpi = 300, bg = "transparent"
  )
  p
})()


**Take-away**

**Early impact, short window.** Direct air connectivity is **strongly** associated with excess mortality during the first wave (ρ ≈ 0.55 in 2020), implying that early import pressure mattered.

**Rapid attenuation.** The link collapses in 2021 and flips negative in 2022–23, once domestic factors—demography, public-health response, health-system capacity, and vaccination—dominate outcomes.

**Policy implication.** Border closures can delay seeding but have little leverage once community transmission is established; long-run mortality depends on internal mitigation and resilience.

> @warnocksmith2021 show that nimble Chinese LCCs—**especially Spring Airlines**—pivoted almost entirely to domestic, price-sensitive demand and recovered first. Such domestic-heavy networks generate **lots of flights but minimal cross-border exposure**, providing a behavioural mechanism for the **negative Spearman ρ after 2021** (Figure @rho_timeseries).

## Snapshot scatter (05 May 2020)

In [ ]:
p20 <- corr_df |>
  filter(target_date == as.Date("2020-05-05")) |>
  ggplot(aes(
    total_inbound_flights_combined,
    excess_mortality_cumulative_per_million
  )) +
  geom_point(size = 3, alpha = .8, colour = "#009E73") +
  geom_text_repel(aes(label = iso3), max.overlaps = 12, size = 3) +
  scale_x_log10(labels = scales::comma) +
  scale_y_continuous(labels = scales::comma) +
  labs(
    x = "Direct scheduled passenger flights (log-scale)",
    y = "Excess deaths / million",
  )

p20


**Interpretation:**

Countries with the highest direct air connectivity to China and Hong Kong (such as Germany, the UK, France, and the Netherlands) experienced some of the highest excess mortality rates during the first wave of COVID-19. This positive rank correlation (ρ = 0.53) supports the hypothesis that early exposure via direct flights contributed significantly to cross-country differences in first-wave mortality. Outliers (e.g., Sweden, Ireland, Slovakia) may reflect unique domestic factors.

## pp scatter (Inb flights, dec 2019 + mar 2020)

In [ ]:
snapshots <- corr_df |>
  filter(target_date == as.Date("2020-05-05")) |>
  select(
    iso3, excess_mortality_cumulative_per_million,
    flights_dec19 = total_inbound_flights_dec19,
    flights_mar20 = total_inbound_flights_mar20
  ) |>
  pivot_longer(
    starts_with("flights_"),
    names_to  = "month",
    values_to = "inbound_flights"
  ) |>
  mutate(
    month = factor(
      recode(month,
        flights_dec19 = "Dec 2019",
        flights_mar20 = "Mar 2020"
      ),
      levels = c("Dec 2019", "Mar 2020")
    )
  ) |>
  filter(inbound_flights > 0) # drop zero-flight countries

stats_lbl <- snapshots |>
  group_by(month) |>
  summarise(
    rho = cor(inbound_flights,
      excess_mortality_cumulative_per_million,
      method = "spearman"
    )
  ) |>
  mutate(
    label = sprintf("ρ ≈ %.2f", rho),
    x = max(snapshots$inbound_flights) * 0.9,
    y = min(snapshots$excess_mortality_cumulative_per_million) * 0.9
  )

gg_pp <- ggplot(
  snapshots,
  aes(inbound_flights, excess_mortality_cumulative_per_million)
) +
  geom_point(size = 3, alpha = 0.8, colour = "#009E73") +
  geom_text_repel(
    data = \(d) d |> filter(iso3 %in% c("ESP", "ITA", "GBR", "DEU", "BEL", "FRA", "NLD")),
    aes(label = iso3),
    size = 3
  ) +
  geom_text(
    data = stats_lbl,
    aes(x = x, y = y, label = label),
    hjust = 1, vjust = 0, size = 3, fontface = "bold",
    colour = "grey30"
  ) +
  scale_x_log10(
    breaks = c(1, 3, 10, 30, 100, 300, 1000),
    labels = scales::comma,
    expand = expansion(mult = c(.05, .02))
  ) +
  scale_y_continuous(
    labels = scales::comma,
    expand = expansion(mult = .03)
  ) +
  facet_wrap(~month, nrow = 1) +
  labs(
    x = "Direct scheduled passenger flights (log-scale)",
    y = "Excess deaths / million (to 5 May 2020)"
  ) +
  theme_minimal(base_size = 11) +
  theme(
    strip.text    = element_text(face = "bold"),
    panel.spacing = grid::unit(1, "cm")
  )

gg_pp


**Interpretation**

**Consistent moderate association.** In both December 2019 and March 2020, countries with more direct scheduled flights from China/Hong Kong tended to record higher excess mortality by 5 May 2020 (Spearman ρ ≈ 0.50–0.57).

**Driven by a handful of hubs.** The positive tail is dominated by major Western-European gateways—Spain, Italy, the UK, Belgium, France and the Netherlands which were among the first and hardest-hit countries.

**Why the right-skew?** Secondary China-Europe links such as **PEK–MUC** collapsed to ≈ 2 % of their 2017 revenue by March 2020, whereas the trunk **HKG–LHR** sector still moved ≈ 13 000 passengers in September 2020 \[@warnocksmith2021\]. This divergence creates the long horizontal tail in **Figure @scatter_dec_mar** (right panel), where just a few hubs sit far out on the x-axis and “pull” the correlation.

**Limits of air-traffic seeding.** The pattern supports the hypothesis that early international connectivity helped seed Europe”s first wave. Yet not every highly connected country suffered high mortality, and conversely some hard-hit countries were not major hubs—pointing to domestic factors (age structure, public-health response, health-system capacity) that modulate outcomes.

**Time-bound effect.** After 2020 the association fades and even reverses, consistent with evidence that border controls matter mainly before widespread community transmission is established.

## Caterpillar plot

In [ ]:
pal <- c(
  "Dec 2019" = "#E69F00",
  "Mar 2020" = "#56B4E9",
  "Combined" = "#009E73"
)

gg_cater <- spearman_res |>
  filter(variable %in% c("Dec 2019", "Mar 2020", "Combined")) |>
  mutate(variable = factor(variable,
    levels = c("Combined", "Dec 2019", "Mar 2020")
  )) |>
  ggplot(aes(year, rho, colour = variable)) +
  geom_hline(yintercept = 0, linetype = "dashed", colour = "grey30") +
  geom_errorbar(
    aes(ymin = ci_lo, ymax = ci_hi),
    width = .15, linewidth = .9,
    position = position_dodge(width = .4)
  ) +
  geom_point(size = 3, position = position_dodge(width = .4)) +
  scale_colour_manual(values = pal, name = NULL) +
  labs(x = NULL, y = "Spearman ρ (± 95 % CI)") +
  theme_minimal(base_size = 11) +
  theme(legend.position = "top")

gg_cater


**Interpretation:**

All three exposure metrics are clearly positive in 2020; in 2021 their CIs straddle zero, and from 2022 the combined metric is significantly negative, confirming a genuine sign-flip rather than sampling noise.

# Limitations

Can be used for Thesis.qmd

## Data limitations

**Flight-exposure proxy (EUROCONTROL, IFR plans).** Our exposure estimates count ICAO flight types S (scheduled) and N (non-scheduled commercial) as reported in EUROCONTROL’s IFR data.

-   Actual passenger numbers per flight are unknown; all filed flights—including “ghost” (very low-occupancy) and cargo-only flights—are counted equally.
-   Itineraries with connecting segments via third-country hubs (e.g., DXB, DOH) are not fully captured, so exposure is under-estimated for transfer states and may be over-estimated for major cargo airports such as Liège.
-   In 2020, Liège Airport handled over 1.1 million metric tons of airfreight (ranking 6th in Europe), with cargo volumes rising 23.5% year-on-year despite the pandemic—contrasting with heavy declines in passenger traffic at most airports \[@fraport2021\]. This highlights the distinct behavior of cargo airports, where flight counts and exposure indices remained elevated due to surging demand for freight and widespread use of “preighter” operations (passenger planes repurposed for cargo).
-   Only four “representative” months (Mar, Jun, Sep, Dec) are available per year from EUROCONTROL’s data releases \[@eurocontrolData2025\], potentially missing key seeding windows (February 2020).

This limitation is directly reflected in our results: for example, Belgium”s exposure index does not drop in line with other countries during the pandemic period (**see Figure @pct_change_filtered**), consistent with Liège”s unique role as a freight hub.

**Ghost/Abnormal flights (load-factor bias).** The phenomenon of “ghost flights” — nearly empty aircraft operated mainly to preserve slots — is well documented in Europe \[@sun2022_b\]. @sun2022_b show that such flights peaked in March–April 2020, especially among low-cost carriers and on dense intra-European routes.

-   Our exposure metric cannot see passenger loads, so it counts ghost, cargo-only, and regularly filled flights equally; exposure is therefore overstated whenever ghost flights dominate a route.
-   @sun2022_b focus on intra-EU sectors, but similar slot-retention economics appeared on some **China–Europe trunk links**. For example, @warnocksmith2021 find that **on residual HKG–LHR services average Y-class fares jumped from ≈ US\$ 772 (Feb 2020) to US\$ 1 052 in March while loads collapsed** - a classic ghost-flight pattern.
-   Because cargo and ghost flights both keep IFR plans “alive,” any flight-count proxy will exaggerate true person-to-person exposure on affected routes; this bias is most acute at slot-controlled hubs.

**Passenger vs. Cargo Dynamics: Exposure Index Bias at Cargo Hubs.** A core limitation of our flight-exposure proxy is that it cannot distinguish between passenger, cargo, and “ghost” flights (near-empty for slot retention), especially at major hubs. While most European airports saw catastrophic declines in passenger traffic during 2020 (e.g., Frankfurt -74%, \[@fraport2021\]), cargo hubs like Liège (+23.5% cargo volume year-over-year (YoY)) or Frankfurt (-8.3% cargo) showed much greater resilience—partly due to a surge in dedicated freighters and “preighter” (cargo-only) flights. At Frankfurt, over 8,600 preighter flights accounted for 8% of annual cargo, and freighters made up 80% of cargo movements.

Consequently, flight counts and exposure indices may significantly overstate real human exposure at major cargo airports. This explains why indices for Belgium and Germany remained high despite catastrophic drops in passenger travel (see Figure @pct_change_filtered)—a pattern also seen at other major cargo hubs. Such discrepancies highlight the importance of careful interpretation when relying on flight-based proxies for pandemic exposure.

**Two-month window (Dec 2019 & Mar 2020).** EUROCONTROL’s public archive releases only four sample months per year; February—a critical seeding month—is missing \[@eurocontrolData2025\].

**Excess-mortality uncertainty.** Outcome data merge the World Mortality Dataset with HMD-STMF. National registration lags, baseline-fit choices, and wartime disruption (Ukraine ≥ 2022) widen credible intervals \[@karlinsky2021; @msemburi2023\]. Notably, @msemburi2023. caution that “basing rankings on point estimates will often be misleading” and discuss the “hazards of ranking countries’ epidemic responses” given large data and model uncertainties.

– A ± 7-day tolerance around 5 May grabs the nearest non-NA value (day_diff ≤ 7); this reduces missingness but adds timing noise. – Country ranks are more stable than absolute levels, yet still volatile for very small populations. – As emphasized by @msemburi2023, “hazards of ranking countries” epidemic responses based solely on excess-mortality estimates should be acknowledged.”

-   **Population denominator.** OWID”s mid-2020 snapshot (UN WPP 2024–derived) differs from raw WPP by \< 5 % in 38 / 41 countries; the largest gaps (Cyprus –31 %, Ukraine –11 %, North Macedonia +11 %) stem from boundary or census updates. Re-running all correlations with WPP figures shifts Spearman ρ by **\< 0.01**, so OWID values were retained for consistency.

### Methodological

-   **Ecological, country-level design.** Results are based on national aggregates, which mask substantial sub-national variation in both air-traffic exposure and health outcomes (e.g. Lombardy vs. Sicily in Italy).

– This approach risks the ecological fallacy: relationships observed at the country level may not hold at the city, regional, or individual level. – Therefore, findings should not be interpreted as causal or representative for specific regions or population subgroups.

**Small-N, rank-based correlation.** With ≈ 25 observations, Spearman ρ is sensitive to ties and outliers; one hub country can swing the estimate. We report 5 000-replicate bootstrap CIs, but sampling error remains large \[@diciccio1996\].

**Residual confounding.** Excess-mortality differences across countries are strongly shaped by vulnerability factors—GDP, poverty, and income inequality. We do **not** adjust for these, so findings may reflect socioeconomic risk more than direct effects of air connectivity \[@ioannidis2023, @msemburi2023\].

### Causal interpretation

**Temporal ordering helps but does not prove causality.** The **December-2019** exposure was measured months **before** COVID-19 deaths peaked in Europe, so reverse causation (high mortality → more flights) is implausible. By contrast, the **March-2020** metric is already affected by border closures, so its association may mix cause and effect. Because all exposure measures precede the 5 May mortality snapshot, reverse causation (i.e. deaths causing flight reductions) is implausible for December and mitigated—but not eliminated—for March.

**Residual confounding is unavoidable.** Country-level correlates—age structure, health-system capacity, NPIs, socio-economic status—remain uncontrolled and could drive part of the relationship.

**External validity is narrow.** Results pertain to continental EUROCONTROL members during the Wuhan/Alpha era. Islands, later variants, or regions with different mobility patterns may behave differently.

> **Bottom line.** This is a *hypothesis-generating* study: early direct flights likely contributed to first-wave seeding, but their statistical signal vanishes once community transmission dominates. Data gaps, ecological design, and unmeasured confounders limit strong policy claims about long-run border-control effectiveness.

## Take-away

> **Early seeding signal, rapidly eclipsed.**  
> Direct flights from China / Hong Kong explain roughly one-quarter of the *cross-country* spread in first-wave excess mortality (May 2020). That signal fades to zero in 2021 and reverses thereafter, once community transmission and domestic counter-measures dominate outcomes.

# Appendix

In [ ]:
# constant and helpers
REF <- as.Date("2020-05-05")
boot_rho <- function(d, idx) {
  with(
    d[idx, ],
    cor(total_inbound_flights_combined,
      excess_mortality_cumulative_per_million,
      method = "spearman"
    )
  )
}

# countries with complete series
FULL_SERIES <- corr_df |>
  count(iso3) |>
  filter(n == 4) |>
  pull(iso3)

# slice 2020 and compute
snap_2020 <- corr_df |>
  filter(
    target_date == REF,
    iso3 %in% FULL_SERIES
  )

set.seed(42)
boot_out <- boot::boot(snap_2020, boot_rho, R = 5000)
ci <- quantile(boot_out$t, c(.025, .975), na.rm = TRUE)

result_tbl <- tibble::tibble(
  rho  = round(boot_out$t0, 3),
  ci   = sprintf("[%.3f , %.3f]", ci[1], ci[2]),
  n    = nrow(snap_2020)
)

knitr::kable(
  result_tbl,
  caption = "Spearman $\\rho$ for the 2020 snapshot **restricted to the 25 countries that have a complete four-year excess-mortality series**."
)


      rho ci                     n
  ------- ------------------- ----
    0.526 \[0.177 , 0.771\]     25

  : Spearman $\rho$ for the 2020 snapshot **restricted to the 25
  countries that have a complete four-year excess-mortality series**.
